## Algorithm: Transformers including data engineering
#### Dataset: SelfRegulationSCP2
#### Author: Edgar Acuna
#### Created:  Augist 20, 2024.

In [ ]:
# cnn lstm model
import numpy as np
from numpy import mean
from numpy import std, unique
from numpy import dstack
from pandas import read_csv
from sklearn.model_selection import train_test_split
from matplotlib import pyplot
 

In [ ]:
import pandas as pd
from aeon.datasets import load_classification
from sklearn import preprocessing

import pandas as pd
from aeon.datasets import load_classification
from sklearn import preprocessing

def load_data(dataset, split="Train"):
    if split=="Train":
        X, y = load_classification(dataset, split="Train")
    else:
        X, y = load_classification(dataset, split="Test")
    print(" Shape of X = ", X.shape)
    print(" Shape of y = ", y.shape)
    #Swap dimensions so features are the third dimension
    X = X.swapaxes(1, 2)
    print(" New shape of X = ", X.shape)
    #prepare y
    y = pd.DataFrame(y)
    enc = preprocessing.OneHotEncoder(dtype=int)
    enc.fit(y)
    y = enc.transform(y).toarray()
    print(" New shape of y = ", y.shape)
    return X, y


In [ ]:
# loading Training data
X, y = load_data("SelfRegulationSCP2",split="Train")

In [ ]:
#class size in training 
import numpy as np
np.sum(y,axis=0)

In [ ]:
import numpy as np
yo=np.argmax(y,axis=1)
yo=np.array(yo,dtype=int)
#print(yo)

In [ ]:
X1=X[yo==0]
X2=X[yo==1]

In [ ]:
Xa1=X1.mean(axis=0).T
Xa1=pd.DataFrame(Xa1)
Xa2=X2.mean(axis=0).T
Xa2=pd.DataFrame(Xa2)
print(Xa2.shape)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
fig, ax = plt.subplots(figsize=(20,10))
ejex=range(0,1152,16)
fig.suptitle("Fig. Plot of mean values by channel in each group-SCP2 training dataset",fontsize=16)
ax.plot(ejex, Xa1.iloc[0,ejex], color='black',lw=5, label='G1')
ax.plot(ejex,Xa1.iloc[1,ejex], color='red', lw=5, label='G1')
ax.plot(ejex, Xa1.iloc[2,ejex], color='blue', lw=5, label='G1')
ax.plot(ejex, Xa1.iloc[3,ejex], color='green',lw=5,  label='G1')
ax.plot(ejex,Xa1.iloc[4,ejex], color='cyan', lw=5, label='G1')
ax.plot(ejex, Xa1.iloc[5,ejex], color='magenta', lw=3,  label='G1')
ax.plot(ejex, Xa1.iloc[6,ejex], color='yellow', lw=3, label='G1')
ax.plot(ejex, Xa2.iloc[0,ejex], color='black',lw=3, label='G2')
ax.plot(ejex,Xa2.iloc[1,ejex], color='red', lw=3, label='G2')
ax.plot(ejex, Xa2.iloc[2,ejex], color='blue', lw=3, label='G2')
ax.plot(ejex, Xa2.iloc[3,ejex], color='green',lw=3,  label='G2')
ax.plot(ejex,Xa2.iloc[4,ejex], color='cyan', lw=3, label='G2')
ax.plot(ejex, Xa2.iloc[5,ejex], color='magenta', lw=3, label='G2')
ax.plot(ejex, Xa2.iloc[6,ejex], color='yellow', lw=3, label='G2')
ax.legend(loc='best', frameon=False)

In [ ]:
#boxplot of the features in the training set
np.sum(y,axis=0)
tempotr=X.reshape(X.shape[2],-1).T
tempotr=pd.DataFrame(tempotr)
pyplot.boxplot(tempotr)
pyplot.show()

In [ ]:
# loading Testing data
Xt, yt = load_data("SelfRegulationSCP2",split="Test")

In [ ]:
#class size in testing
import numpy as np
np.sum(yt,axis=0)

In [ ]:
a=np.sum(yt,axis=0)
np.max(a/Xt.shape[0])

In [ ]:
#Boxplot of the features in the testing dataset
tempote=Xt.reshape(Xt.shape[2],-1).T
tempote=pd.DataFrame(tempote)
pyplot.boxplot(tempote)
pyplot.show()

In [ ]:
tempotr.describe()

### The features are not so relevant, since the mean values by group are quite similar. 

In [ ]:
tempote.describe()

In [ ]:
import numpy as np
yo=np.argmax(y,axis=1)
yo=np.array(yo,dtype=int)
#print(yo)

In [ ]:
#Boxplots of each feature in the groups pf the training dataset
temptr=pd.DataFrame(tempotr)
from itertools import cycle
variables= cycle(yo)
print(variables)
temptr['group'] = [next(variables) for variable in range(len(temptr))]
#Bboxplot  of features by class
temptr.boxplot(by='group')

In [ ]:
temptr.groupby('group').mean()

### Distances

In [ ]:
#Distance between the features in the two groups of the treining
means=temptr.groupby('group').mean()
from scipy.spatial.distance import pdist, squareform

distances = pdist(means, metric='euclidean')
distances

In [ ]:
tempoA=X.reshape(X.shape[1],-1).T
print(tempoA.shape)

In [ ]:
#media total training
meanTr=tempoA.mean(axis=0)
meanTr.shape

In [ ]:
tempoB=pd.DataFrame(tempoA)
from itertools import cycle
variables= cycle(yo)
print(variables)
tempoB['group'] = [next(variables) for variable in range(len(tempoB))]
#Bboxplot  of features by class
a=tempoB.groupby('group').mean()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
fig, ax = plt.subplots(figsize=(20,10))
ejex=range(0,1152,16)
fig.suptitle("Fig. Plot of mean values of all time series by group-SCP2 training dataset",fontsize=16)
ax.plot(ejex, a.iloc[0,ejex], color='red',lw=5, label='G1')
ax.plot(ejex,a.iloc[1,ejex], color='blue', lw=5, label='G2')
ax.legend(loc='best', frameon=False)

### Feature Selection

In [ ]:
#Feature selection
from sklearn.feature_selection import  mutual_info_classif, SelectPercentile, f_classif
ym=tempoB['group']
Xm=tempoB.iloc[:,0:1152]
scores = mutual_info_classif(Xm,ym)
#scores /= scores.max()
importance=[i for i,v in enumerate(scores) if v > .5]
len(importance)*100/1152

In [ ]:
selector = SelectPercentile(f_classif, percentile=30)
Xm_new=selector.fit_transform(Xm, ym)
scores = -np.log10(selector.pvalues_)
#scores /= scores.max()
#print(scores)
#critical p-value=.10
importance=[i for i,v in enumerate(scores) if v > 1]
len(importance)*100/1152

In [ ]:
tempote.describe()

In [ ]:
import numpy as np
yot=np.argmax(yt,axis=1)
yot=np.array(yot,dtype=int)

In [ ]:
#Boxplots of each feature in the groups pf the testing dataset
tempte=pd.DataFrame(tempote)
from itertools import cycle
variables= cycle(yot)
print(variables)
tempte['group'] = [next(variables) for variable in range(len(tempte))]
#Bboxplot  of features by class
tempte.boxplot(by='group')

In [ ]:
import numpy as np
Xb= np.concatenate((X, Xt),axis=0)
print(Xb.shape)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
#print(tempo.shape)
X = scaler.fit_transform(X.reshape(X.shape[2], -1)).reshape(X.shape)
Xt = scaler.fit_transform(Xt.reshape(Xt.shape[2], -1)).reshape(Xt.shape)
Xb = scaler.fit_transform(Xb.reshape(Xb.shape[2], -1)).reshape(Xb.shape)
X1 = X.reshape(X.shape[2],-1).T
#print(X)
tempodf=pd.DataFrame(X1)
print(tempodf.shape)
pyplot.boxplot(tempodf)
pyplot.show()

In [ ]:
tempoA1=Xt.reshape(Xt.shape[1],-1).T
print(tempoA1.shape)

In [ ]:
#media total testung
meanTe=tempoA1.mean(axis=0)
meanTe.shape

In [ ]:
#Distance Training and Testsing
from scipy.spatial import distance
dst = distance.euclidean(meanTr, meanTe)
dst

In [ ]:
#Dickey-Fuller Test
from statsmodels.tsa.stattools import adfuller
Xest = tempoB.iloc[150,0:1152]
result = adfuller(Xest)
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))

In [ ]:
sm.graphics.tsa.plot_acf(Xest, alpha=.05)